# Import Statements

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout, Input
import os

2025-07-05 11:20:22.451173: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-05 11:20:22.749613: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-05 11:20:22.918126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751714423.197781    3198 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751714423.290765    3198 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751714423.943551    3198 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
print(tf.__version__)

2.19.0


# Generate synthetic data and save as CSV if not present

In [3]:
csv_path = "health_data.csv"
if not os.path.exists(csv_path):
    num_days = 120
    np.random.seed(42)
    dates = pd.date_range(end=pd.Timestamp.today(), periods=num_days)
    df_synth = pd.DataFrame({
        'date': dates,
        'HRV': np.random.normal(50, 10, num_days),
        'RHR': np.random.normal(60, 5, num_days),
        'Sleep': np.random.normal(7, 1, num_days),
        'Steps': np.random.normal(8000, 2000, num_days),
        'Mood': np.random.normal(3, 1, num_days),
        'Readiness': np.random.normal(75, 10, num_days)
    })
    df_synth.to_csv(csv_path, index=False)
    print(f"Synthetic data generated and saved to {csv_path}")

# Load and preprocess data

In [4]:
df = pd.read_csv(csv_path, parse_dates=['date'])
df = df.sort_values('date')

feature_cols = ['HRV', 'RHR', 'Sleep', 'Steps', 'Mood']
target_col = 'Readiness'

df[feature_cols] = (df[feature_cols] - df[feature_cols].mean()) / df[feature_cols].std()

# Create sequences for model input

In [5]:
sequence_length = 7
X = []
y = []

for i in range(len(df)-sequence_length):
    seq = df[feature_cols].iloc[i:i+sequence_length].values
    label = df[target_col].iloc[i+sequence_length]
    X.append(seq)
    y.append(label)

X = np.array(X)
y = np.array(y)

print(f"Shape of input X: {X.shape}")
print(f"Shape of labels y: {y.shape}")
    

Shape of input X: (113, 7, 5)
Shape of labels y: (113,)


# 3. Build the Conv1D model (TFLite-compatible)

In [6]:
def build_readiness_model(seq_len, num_features):
    model = Sequential([
        Input(shape=(seq_len, num_features), name="input"),
        Conv1D(32, kernel_size=3, activation='relu', padding='same'),
        GlobalAveragePooling1D(),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = build_readiness_model(sequence_length, len(feature_cols))
model.summary()

2025-07-05 11:20:27.259361: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 7, 32)          │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,601 (6.25 KB)

 Trainable params: 1,601 (6.25 KB)

 Non-trainable params: 0 (0.00 B)

# Train the model

In [7]:
history = model.fit(
    X, y,
    epochs=30,
    batch_size=8,
    validation_split=0.2,
    verbose=2
)

Epoch 1/30
12/12 - 1s - 101ms/step - loss: 5652.4355 - mae: 74.4255 - val_loss: 5505.0381 - val_mae: 73.5276
Epoch 2/30
12/12 - 0s - 5ms/step - loss: 5576.9927 - mae: 73.9174 - val_loss: 5440.9316 - val_mae: 73.0908
Epoch 3/30
12/12 - 0s - 6ms/step - loss: 5499.1743 - mae: 73.3837 - val_loss: 5361.6597 - val_mae: 72.5475
Epoch 4/30
12/12 - 0s - 5ms/step - loss: 5380.0298 - mae: 72.5805 - val_loss: 5265.9360 - val_mae: 71.8860
Epoch 5/30
12/12 - 0s - 6ms/step - loss: 5257.6348 - mae: 71.7288 - val_loss: 5148.1567 - val_mae: 71.0637
Epoch 6/30
12/12 - 0s - 6ms/step - loss: 5096.1538 - mae: 70.6049 - val_loss: 4997.7314 - val_mae: 69.9993
Epoch 7/30
12/12 - 0s - 6ms/step - loss: 4911.0503 - mae: 69.2439 - val_loss: 4810.6831 - val_mae: 68.6523
Epoch 8/30
12/12 - 0s - 6ms/step - loss: 4655.1621 - mae: 67.3802 - val_loss: 4590.2725 - val_mae: 67.0301
Epoch 9/30
12/12 - 0s - 12ms/step - loss: 4379.3569 - mae: 65.2936 - val_loss: 4331.4185 - val_mae: 65.0714
Epoch 10/30
12/12 - 0s - 5ms/step 

# Export to Tensorflow Lite model

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("readiness_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as readiness_model.tflite")

INFO:tensorflow:Assets written to: /tmp/tmp0kt46gwb/assets


INFO:tensorflow:Assets written to: /tmp/tmp0kt46gwb/assets


Saved artifact at '/tmp/tmp0kt46gwb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7, 5), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  133877916971280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133877916973584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133877916974544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133877916974352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133877916970896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133877916972816: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1751714431.240105    3198 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1751714431.240133    3198 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-07-05 11:20:31.240533: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp0kt46gwb
2025-07-05 11:20:31.241040: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-05 11:20:31.241049: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp0kt46gwb
I0000 00:00:1751714431.244242    3198 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-07-05 11:20:31.244878: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-05 11:20:31.269627: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp0kt46gwb
2025-07-05 11:20:31.276159: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

TFLite model saved as readiness_model.tflite
